### Custom ML model to identify local similarities ("links") with sequence profiles

Let $$X \in \{0,1\}^{B\times N\times 6 \times T \times 21}$$ be a batch of **one-hot encoded input translated sequences**,
where $B$ is `batch_size`, $N$ is the number of genomes and $T$ is the `tile_size` (in aa).
The 6 is here the number of translated frames in order (0,+),(1,+),(2,+),(0,-),(1,-),(2,-).
The 21 is here the size of the considered amino acid alphabet.


In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import tensorflow as tf
import logomaker
from time import time
from tqdm import tqdm
from IPython.display import Audio

import importlib
import sequtils as su
print(importlib.reload(su))
import seq
print(importlib.reload(seq))
import dataset as dsg
print(importlib.reload(dsg))
import model
print(importlib.reload(model))

print(tf.__version__)

%load_ext memory_profiler

<module 'sequtils' from '/home/jovyan/brain/genomegraph/learn_specific_profiles/sequtils.py'>
<module 'seq' from '/home/jovyan/brain/genomegraph/learn_specific_profiles/seq.py'>
<module 'dataset' from '/home/jovyan/brain/genomegraph/learn_specific_profiles/dataset.py'>
<module 'model' from '/home/jovyan/brain/genomegraph/learn_specific_profiles/model.py'>
2.5.0


In [2]:
use_real_data = False

### Create random genomes as toy data 

In [3]:
N = 8            # number of genomes
tile_size = 334  # tile size measured in amino acids
# A tile is a consecutive subsequence of _one_ contig/scaffold/chromosome.
# Tiles should be about gene-sized.

#genome_sizes = [[210,100], [30,220,150], [230,110,120,90], [180]] # in nucleotides
#insertPatterns = ["ATGATGATG", "CCCCCCCCCCCC"]
#genomes = seq.getRandomGenomes(N, genome_sizes, insertPatterns, mutationProb=0.2, verbose=True)

genome_sizes = [[10000]] * N
                # in nucleotides
insertPatterns = ["ATGGCAAGAATTCAATCTACTGCAAATAAAGAA"] 
repeatPatterns = ['AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA']
genomes, repeatTracking, insertTracking = seq.getRandomGenomes(N, genome_sizes, insertPatterns,
                                                               repeatPatterns,
                                                               mutationProb=0.0, 
                                                               repeatMultiple=range(1,2),
                                                               repeatInsert=range(10,11),
                                                               verbose=True)
print("insert:", insertTracking)
print("repeat:", repeatTracking)

Pattern AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA translates to  ['REPEATAEPER', 'ENLKLLLNLK', 'RT*SYC*T*K', 'SFRFSSSFRFS', 'LSGSAVASGS', 'FQVQQ*LQVL']
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 contig 0 at position 9422
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 contig 0 at position 8432
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 contig 0 at position 6250
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 contig 0 at position 3996
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 contig 0 at position 3624
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 contig 0 at position 8390
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 contig 0 at position 9193
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 contig 0 at position 4802
  mutated to AGAGAACCTGAAGCTACTGCTGAACCTGAAAGA and inserted in genome 0 

In [4]:
if True:
    flat_genomes = []
    tracking = {}
    i = 0
    for g in range(len(genomes)):
        for c in range(len(genomes[g])):
            flat_genomes.append(SeqRecord(Seq(genomes[g][c]),
                                          id=str(i), description=""))
            tracking[i] = {'insert': dict(insertTracking[g][c]),
                           'repeat': dict(repeatTracking[g][c])}
            i += 1
            
    with open("artificial.fasta", "w") as output_handle:
        SeqIO.write(flat_genomes, output_handle, "fasta")
        
    with open("tracking.json", "w") as fh:
        json.dump(tracking, fh)

In [5]:
def printExpectedPatterns():
    desiredPatternAA, repeatPatternAA = None, None
    if insertPatterns is not None:
        desiredPatternAA = []
        for pattern in insertPatterns:
            desiredPatternAA.extend(su.six_frame_translation(pattern))

        print("Desired:", desiredPatternAA)

    if repeatPatterns is not None:
        repeatPatternAA = []
        for pattern in repeatPatterns:
            repeatPatternAA.extend(su.six_frame_translation(pattern))

        print("Repeat:", repeatPatternAA)
        
    return desiredPatternAA, repeatPatternAA

desiredPatternAA, repeatPatternAA = printExpectedPatterns()

Desired: ['MARIQSTANKE', 'WQEFNLLQIK', 'GKNSIYCK*R', 'FFICSRLNSCH', 'SLFAVD*ILA', 'LYLQ*IEFLP']
Repeat: ['REPEATAEPER', 'ENLKLLLNLK', 'RT*SYC*T*K', 'SFRFSSSFRFS', 'LSGSAVASGS', 'FQVQQ*LQVL']


---

#### Interlude: Magic Commands

https://towardsdatascience.com/speed-up-jupyter-notebooks-20716cbe2025

`%time command` prints runtime of command  
`%prun command` profiler for command, what steps take how much time? Adds overhead  
`%memit command` peak memory usage and memory increment  
`%mprun command` memory profiling

---

In [6]:
# Load Real Data
if use_real_data:    
    files = ['hg38.fa', 'mm10.fa', 'hetGla2.fa', 'macFas5.fa']
    genomes = [[] for _ in range(len(files))]
    seqnames = [[] for _ in range(len(files))]
    datapath = "../data/20210517_flanked/subset"
    def loadRealGenomes(genomes=genomes):
        for i in range(len(files)):
            genomes[i].extend([str(seq.seq) for seq in SeqIO.parse(os.path.join(datapath, files[i]), 'fasta')])
            seqnames[i].extend([str(seq.id) for seq in SeqIO.parse(os.path.join(datapath, files[i]), 'fasta')])
            
    %memit loadRealGenomes()
    
    # load pickled Q to safe time
    Qfile = os.path.join(datapath, "backgroundAAFreqs.pkl4")
    if not os.path.isfile(Qfile):
        print("Getting Q")
        %time Q = seq.backGroundAAFreqs(genomes, True)
        with open(Qfile, 'wb') as fh:
            pickle.dump(Q, fh, protocol=4)
            
    else:
        with open(Qfile, 'rb') as fh:
            Q = pickle.load(fh)
    
else:
    %time Q = seq.backGroundAAFreqs(genomes, True)

background freqs:  159968.0 *
C 0.0306
K 0.0315
E 0.0333
W 0.0153
T 0.0619
G 0.0616
Y 0.0299
A 0.0620
I 0.0456
N 0.0315
V 0.0619
H 0.0305
S 0.0963
D 0.0306
F 0.0325
M 0.0150
R 0.0925
L 0.0957
P 0.0616
Q 0.0323
* 0.0479
CPU times: user 50.4 ms, sys: 0 ns, total: 50.4 ms
Wall time: 49.9 ms


For seedFinding data on gs3

    background freqs:  765611260.0 *
    C 0.0329
    K 0.0412
    E 0.0332
    W 0.0165
    T 0.0578
    G 0.0558
    Y 0.0334
    A 0.0510
    I 0.0546
    N 0.0361
    V 0.0578
    H 0.0321
    S 0.0929
    D 0.0280
    F 0.0406
    M 0.0172
    R 0.0710
    L 0.1039
    P 0.0558
    Q 0.0352
    * 0.0527
    CPU times: user 7min 14s, sys: 1.69 s, total: 7min 16s
    Wall time: 7min 16s

In [7]:
print(len(genomes), len(genomes[0]), len(genomes[0][0]))

8 1 10000


In [8]:
# uniform background distribution appears to be rather better
Q = np.ones(21, dtype=np.float32)/21

#### Read in the genome

In [9]:
genomes[0][0][0:100]

'AGACTATGTTGGGAAGGGAGTCACCGTGGTATTCCATAGCTAGTGGTAGGCCGTAGCTCCTGGGCCGCCTTATGATATATCGAGAGCGAGCTAGTTTACA'

In [10]:
dsg.testGenerator(genomes, 5, tile_size, 50000)

[INFO] >>> Test genome lengths: [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]


2it [00:02,  1.16s/it]
0it [00:00, ?it/s]

[INFO] >>> testGenerator - restore sequences: All good


2it [00:16,  8.25s/it]

[INFO] >>> testGenerator - restore positions: All good


In [11]:
genome_sizes = [sum([len(s) for s in genome]) for genome in genomes]
batch_size = 1  # number of X to generate per batch
tiles_per_X = 13 # number of tiles per X (-> X.shape[0])
steps_per_epoch = max(1, np.mean(genome_sizes) // (batch_size*tiles_per_X*tile_size*3))
print(genome_sizes, " -> ", steps_per_epoch)

[10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000]  ->  1


---

#### Plot Functions

In [12]:
def plotLogo(P, idxarray = None, pScores = None, max_print=5, label="score"):
    """
    max_print  print up to this many logos
    """
    dfs = su.makeDFs(P.numpy())
    for i in range(min(P.shape[2],max_print)):
        j = idxarray[i] if idxarray is not None else i
        profile_df = dfs[j]
        logo = logomaker.Logo(profile_df, vpad=.1, width=1)
        logo.style_xticks(anchor=0, spacing=1, rotation=45)
        logo.ax.set_ylabel('information (bits)')
        logo.ax.set_title(f"Profile {j}" + (f" {label}={pScores[j]:.3f}") if pScores is not None else "")

In [13]:
# plot the training history as loss and accuracy curves
def plotHistory(history):
    loss = history['loss']
    Rmax = history['Rmax']
    Rmin = history['Rmin']
    Smax = history['Smax']
    Smin = history['Smin']
    epochs = range(1, len(loss) + 1)

    fig, ax = plt.subplots(ncols = 2, figsize = (15, 6))
    ax[0].plot(epochs, loss, 'bo', label = 'Training loss')
    ax[0].set_title('Training loss')
    ax[0].legend()

    ax[1].plot(epochs, Rmax, 'bo', label = 'Rmax')
    ax[1].plot(epochs, Rmin, 'b+', label = 'Rmin')
    ax[1].plot(epochs, Smax, 'go', label = 'Smax')
    ax[1].plot(epochs, Smin, 'g+', label = 'Smin')
    ax[1].set_title('Training R and S')
    ax[1].legend();
    
    return fig, ax

## TensorFlow Model
Let $$P = (P[w,c,u]) \in [0,1]^{k \times 21 \times U}$$
be a collection of $U$ amino acid **profiles**, each of length $k$.
Let $$ Q = (Q[c]) \in [0,1]^{21}$$ be a background amino acid distribution.

Both are normalized distributions:
$$ \sum_c P[w,c,u] = \sum_c Q[c] = 1 \qquad \forall u,w.$$

Define the scores tensor 
$$ S \in \mathbb{R}^ {B\times N \times U}$$
by
$$ S[b,g,u] = \max_{f=0}^5 \max_{v=0}^{T-k} \sum_{w=0}^{k-1} \sum_{c=0}^{20} X[b,g,f,v+w,c] \cdot \ln \frac {P[w,c,u]}{Q[c]}.$$

For a given batch $S[b,g,u]$ is the maximal score that the $u$-th profile scores in the $b$-th tile of genome $g$.
It can be computed using a **one dimensional convolution** and max pooling.

Define the intermediate variables:

$R \in [0,1]^{k \times 21 \times U}$ by 
$$ R[w,c,u] := \ln \frac {P[w,c,u]}{Q[c]}.$$

$Z \in \mathbb{R}^{B\times N \times 6 \times T-k-1\times U}$ by
$$Z[b,g,f,v,u] := \sum_{w=0}^{k-1} \sum_{c=0}^{20} X[b,g,f,v+w,c] \cdot R[w,c,u].$$

**Losses:**
Only the best score counts for each frame and tile. Sum them up.
$$L_1 = - \sum_{b=1}^B \sum_{g=1}^G \sum_{u=1}^U S[b,g,u]$$ 

In [14]:
U = 200 # number of profiles to train
k = 11 # length of profiles
alpha = 1e-6 # loss norm
gamma = 0.4  # softmax scale

In [15]:
print(importlib.reload(model))

<module 'model' from '/home/jovyan/brain/genomegraph/learn_specific_profiles/model.py'>


In [16]:
# build and randomly initialize profile model
tf.keras.backend.clear_session() # avoid memory cluttering by remains of old models
specProModel = None
%time %memit specProModel = model.SpecificProfile(k, su.aa_alphabet_size, U, Q, alpha=alpha, gamma=gamma)

peak memory: 1364.37 MiB, increment: 1007.55 MiB
CPU times: user 541 ms, sys: 416 ms, total: 957 ms
Wall time: 1.12 s


### Training Set

In [17]:
#ds_init = dsg.getDataset(genomes, tiles_per_X, tile_size).batch(batch_size).prefetch(3)
#%time P_logit_init = specProModel.seed_P_ds(ds_init)
%time P_logit_init = specProModel.seed_P_genome(genomes)
specProModel.setP_logit(P_logit_init)

#%time P_logit_init = specProModel.seed_P_triplets()
#print(specProModel.units)
#specProModel.setP_logit(P_logit_init)

CPU times: user 9.42 ms, sys: 95 µs, total: 9.51 ms
Wall time: 8.48 ms


---

### See How Perfect Pattern / Repeat Profiles Score w.r.t. Loss Function

In [18]:
runExperiment = False

runExperiment = runExperiment and not use_real_data

In [19]:
if runExperiment:
    import experiment
    print(importlib.reload(experiment))

In [20]:
if runExperiment:
    pProfile = tf.cast( tf.math.log( dsg.oneHot('MARIQSTANKE') + 1e-6 ), tf.float32)
    rProfile = tf.cast( tf.math.log( dsg.oneHot('REPEATAEPER') + 1e-6 ), tf.float32)
    
    # build and randomly initialize profile model
    tf.keras.backend.clear_session() # avoid memory cluttering by remains of old models
    specProModel = None
    specProModel = model.SpecificProfile(k, su.aa_alphabet_size, units=2, Q=Q, alpha=0, gamma=gamma)

    pInit = specProModel._getRandomProfiles()
    pInit[:,:,0] = pProfile.numpy()
    pInit[:,:,1] = rProfile.numpy()
    print(pInit.shape)

    specProModel.setP_logit(pInit)
    print(specProModel.P_logit.shape)

In [21]:
if runExperiment:
    #pscores = experiment.getLossScores(specProModel, 0, genomes, N, tiles_per_X, tile_size, batch_size)
    #rscores = experiment.getLossScores(specProModel, 1, genomes, N, tiles_per_X, tile_size, batch_size)
    pscores = experiment.getLossScores_raw(specProModel, 0, genomes, tiles_per_X, tile_size, batch_size)
    rscores = experiment.getLossScores_raw(specProModel, 1, genomes, tiles_per_X, tile_size, batch_size)
    pscores_raw = experiment.getLossScores_raw(specProModel, 0, genomes, tiles_per_X, tile_size, batch_size, lossNorm=False)
    rscores_raw = experiment.getLossScores_raw(specProModel, 1, genomes, tiles_per_X, tile_size, batch_size, lossNorm=False)
    pscores_unmasked = experiment.getLossScores_raw(specProModel, 0, genomes, tiles_per_X, tile_size, batch_size, maskNonSeqScores=False)
    rscores_unmasked = experiment.getLossScores_raw(specProModel, 1, genomes, tiles_per_X, tile_size, batch_size, maskNonSeqScores=False)
    pscores_raw_unmasked = experiment.getLossScores_raw(specProModel, 0, genomes, tiles_per_X, tile_size, batch_size, maskNonSeqScores=False, lossNorm=False)
    rscores_raw_unmasked = experiment.getLossScores_raw(specProModel, 1, genomes, tiles_per_X, tile_size, batch_size, maskNonSeqScores=False, lossNorm=False)

#### Draw Artificial Genomes and Pattern / Repeat Locations

In [22]:
if runExperiment:
    #bins, vals = experiment.ownHist(pscores.reshape((-1)), 0.00000001)
    bins, vals = experiment.ownHist(pscores_raw, 10)
    experiment.plotOwnHist(bins, vals, precision=1)#, ylim=(0, 30))
    #experiment.plotOwnHist(bins, vals, precision=1, ylim=(0, 30))
    
    bins, vals = experiment.ownHist(pscores, 1)
    print(set([bins[i] for i in range(len(bins)) if vals[i] > 0]))
    experiment.plotOwnHist(bins, vals, precision=1)#, ylim=(0, 30))
    experiment.plotOwnHist(bins, vals, precision=1, ylim=(0, 30))
    
    #bins, vals = experiment.ownHist(pscores.reshape((-1)), 0.00000001)
    bins, vals = experiment.ownHistRel(pscores_raw_unmasked, 10)
    experiment.plotOwnHist(bins, vals, precision=1)#, ylim=(0, 30))
    #experiment.plotOwnHist(bins, vals, precision=1, ylim=(0, 30))
    
    bins, vals = experiment.ownHistRel(pscores_unmasked, 1)
    print(set([bins[i] for i in range(len(bins)) if vals[i] > 0]))
    experiment.plotOwnHist(bins, vals, precision=1)#, ylim=(0, 30))

In [23]:
if runExperiment:
    #bins, vals = experiment.ownHist(pscores.reshape((-1)), 0.00000001)
    bins, vals = experiment.ownHist(rscores_raw, 10)
    experiment.plotOwnHist(bins, vals, precision=1)#, ylim=(0, 30))
    #experiment.plotOwnHist(bins, vals, precision=1, ylim=(0, 30))
    
    bins, vals = experiment.ownHist(rscores, 1)
    print(set([bins[i] for i in range(len(bins)) if vals[i] > 0]))
    experiment.plotOwnHist(bins, vals, precision=1)#, ylim=(0, 30))
    experiment.plotOwnHist(bins, vals, precision=1, ylim=(0, 30))
    
    #bins, vals = experiment.ownHist(pscores.reshape((-1)), 0.00000001)
    bins, vals = experiment.ownHistRel(rscores_raw_unmasked, 10)
    experiment.plotOwnHist(bins, vals, precision=1)#, ylim=(0, 30))
    #experiment.plotOwnHist(bins, vals, precision=1, ylim=(0, 30))
    
    bins, vals = experiment.ownHistRel(rscores_unmasked, 1)
    print(set([bins[i] for i in range(len(bins)) if vals[i] > 0]))
    experiment.plotOwnHist(bins, vals, precision=1)#, ylim=(0, 30))

#### Draw Genomes, Pattern and Repeat Locations, and Loss Scores for each Position

(scores: thin lines; top line: pattern profile (`MARIQSTANKE`), bottom line: repeat profile (`REPEATAEPER`)

In [24]:
if runExperiment:
    #experiment.drawLossScores(pscores, rscores, genomes, N, insertTracking, repeatTracking, tile_size)
    print()

In [25]:
if runExperiment:
    # reset model
    tf.keras.backend.clear_session() # avoid memory cluttering by remains of old models
    specProModel = None
    %time %memit specProModel = model.SpecificProfile(k, su.aa_alphabet_size, U, Q, alpha=alpha, gamma=gamma)

---

### Training

    Training on seedFinding data takes 5.5 h for 10 epochs on greifserv3

In [ ]:
start = time()
specProModel.train(genomes, tiles_per_X, tile_size, 
                   batch_size, steps_per_epoch, epochs=50, 
                   profile_plateau=10, profile_plateau_dev=0.5,
                   verbose_freq=10, n_best_profiles=5, match_score_factor=0.7)
end = time()
print(f"time: {end-start:.2f}")

In [ ]:
plotHistory(specProModel.history)

### Visualize resulting profiles as sequence logos

In [ ]:
if False: # profiles after last report are just seeds
    ds_score = dsg.getDataset(genomes, tiles_per_X, tile_size).batch(batch_size).prefetch(3)
    printExpectedPatterns()
    P = specProModel.getP()
    #pScores = specProModel.max_profile_scores(ds_score)
    #plotLogo(P=P, idxarray = np.argsort(-pScores), pScores=pScores, max_print=20)
    pLosses = specProModel.min_profile_losses(ds_score)
    plotLogo(P=P, idxarray = np.argsort(pLosses), pScores=pLosses, max_print=5, label="loss")

In [ ]:
printExpectedPatterns()
P, Pthresh = specProModel.getP_report()
plotLogo(P=P, pScores=Pthresh, max_print=10, label="Score Threshold")

In [ ]:
# STOP HERE IF `RUN ALL CELLS`
assert False

In [ ]:
def getKmersFromSites(sites, genomes, k, P, Q, epsilon=1e-6):
    kmers = []
    kDNA = k*3
    for site in sites:
        g = site[0].numpy()
        c = site[1].numpy()
        a = site[2].numpy()
        if a < 0:
            kmers.append("")
            continue
            
        b = min(len(genomes[g][c]), a+kDNA)
        f = str(site[4].numpy()) if len(site) >= 5 else ""
        dna = genomes[g][c][a:b]
        dnaUC = dna.upper()
        dnaPre = genomes[g][c][max(0,a-kDNA):a]
        dnaPost = genomes[g][c][b:min(len(genomes[g][c]), b+kDNA)]
        aa = dsg.sequence_translation(dnaUC)
        aaRC = dsg.sequence_translation(dnaUC, True)
        
        # re-calculate score
        if len(aa) == k:
            score = tf.reduce_sum(
                tf.math.multiply( dsg.oneHot(aa),
                                  tf.math.log( tf.maximum( tf.divide(P, Q), epsilon) ) 
                                ) 
            ).numpy()
            scoreRC = tf.reduce_sum(
                tf.math.multiply( dsg.oneHot(aaRC),
                                  tf.math.log( tf.maximum( tf.divide(P, Q), epsilon) ) 
                                ) 
            ).numpy()
        else:
            score = None
            scoreRC = None
        
        kmers.append(dnaPre+" "+dna+" "+dnaPost+"   "+aa+" // "+aaRC+"   "+str(g)+", "+str(c)+", "+f+"   "+str(a)+"   "+str(score)+" // "+str(scoreRC))
        
    return kmers

In [ ]:
# un-mask dataset
ucgenomes = []
for g in range(len(genomes)):
    ucgenomes.append([])
    for c in range(len(genomes[g])):
        ucgenomes[g].append( genomes[g][c].upper() )

In [ ]:
P, Pthresh = specProModel.getP_report()
specProModel.setP_logit(specProModel.getP_report_raw()) # set reported P as profiles, otherwise get matches from randomly chosen seeds
ds_sites = dsg.getDataset(ucgenomes, tiles_per_X, tile_size, True).batch(batch_size).prefetch(3)
sites, scores = specProModel.get_profile_match_sites(ds_sites, 0, 1)
assert len(sites) < 100, print(sites.shape)
idx = tf.reshape( tf.argsort(scores, axis=0, direction='DESCENDING'), (-1))
kmers = getKmersFromSites(sites, genomes, specProModel.k, P[:,:,1], specProModel.Q, specProModel.epsilon)

print(tf.gather(scores ,idx), tf.gather(kmers, idx), sep='\n')

In [ ]:
_,_=printExpectedPatterns()

In [ ]:
dsg.sequence_translation('cagagaacctgaagctactgctgaacctgaaag'.upper())

---

## Profile Score Distribution

In [ ]:
def customHist(distType: str, prec = 1):
    validDistTypes = ['score', 'loss', 'L5score']
    assert distType in validDistTypes, "distType must be one of "+str(validDistTypes)
    
    collect = {}
    ds_score = getDataset().batch(batch_size).prefetch(3)
    for batch in ds_score:
        for X in batch:
            S, _, _ = specProModel(X)
            if distType == 'score':
                for s in np.array(S).flatten():
                    score = np.math.floor(s*prec)
                    if score not in collect:
                        collect[score] = 0
                        
                    collect[score] += 1
                    
            if distType == 'loss':
                _, LpU, _ = specProModel.loss(S)
                for l in LpU:
                    loss = np.math.floor(l*prec)
                    if loss not in collect:
                        collect[loss] = 0
                        
                    collect[loss] += 1
                    
            if distType == 'L5score':
                gamma = .2
                S2 = tf.nn.softmax(gamma*S, axis=0)
                S3 = tf.math.multiply(S, tf.square(S2))
                for s in np.array(S3).flatten():
                    score = np.math.floor(s*prec)
                    if score not in collect:
                        collect[score] = 0
                        
                    collect[score] += 1
                    
    fkeys = [k for k in collect.keys()]
    bins = [b for b in range(min(fkeys), max(fkeys)+1)]
    vals = [collect[k] if k in collect else 0 for k in bins]
    bins = [b/prec for b in bins]
    
    plt.bar(list(range(len(bins))), vals, tick_label=[str(b) for b in bins])
    
    return bins, vals

In [ ]:
bins, vals = customHist('L5score', 0.5)

In [ ]:
bins, vals = customHist('loss', 100)

In [ ]:
bins, vals = customHist('score', 0.1)

     P:     (k, alphSize, U) 
     X:     (tilesPerX, N, 6, T, alphSize) 
     Z:     (tilesPerX, N, 6, T-k+1, U)
     S:     (tilesPerX, N, U)

In [ ]:
Scollect = []
Lcollect = []

if not use_real_data:
    ds_score = getDataset().batch(batch_size).prefetch(150)
    for batch in ds_score:
        for X in batch:
            S, _, Z = specProModel(X)
            _, LpU, _ = specProModel.loss(S)
            Scollect.extend(np.array(S).flatten())
            Lcollect.extend(np.array(LpU).flatten())

In [ ]:
plt.hist(Scollect)

In [ ]:
plt.hist(Lcollect)

In [ ]:
if not use_real_data:
    gamma = .01 # a small value means a more inclusive meaning of near-best
    S2 = tf.nn.softmax(gamma*S, axis=0)
    S3 = tf.reduce_max(S2, axis=0) # the closer to 1, the clearer is the champion match a winner
    S4 = tf.reduce_max(S, axis=0) # ranges over tiles, or soft max like in L1
    S5 = tf.math.multiply(S4, S3) # effectively the best score per genome is divided by the number of matches
    loss_by_unit = tf.reduce_sum(S5, axis=0) / U # sum over genomes
    L5 = tf.reduce_sum(loss_by_unit) # sum over profiles=units
    plt.hist(S4[:,0].numpy().flatten(), bins=20)
    S5.shape

---

### Profile Match Sites

In [ ]:
# un-mask dataset
for g in range(len(genomes)):
    for c in range(len(genomes[g])):
        genomes[g][c] = genomes[g][c].upper()

In [ ]:
ds_sites = dsg.getDataset(genomes, tiles_per_X, tile_size, True).batch(batch_size).prefetch(3)
specProModel.setP_logit(tf.math.log(specProModel.getP_report())) # set reported P as profiles, otherwise get matches from randomly chosen seeds
sites = specProModel.get_profile_match_sites(ds_sites, 8.8)
sites

In [ ]:
plotLogo(P=specProModel.getP_report()[:,:,2:3], max_print=1, label="loss")

In [ ]:
print(importlib.reload(dsg))
print(genomes[2][28][25029:25029+33])
print(dsg.sequence_translation( genomes[2][28][25029:25029+33] ))

In [ ]:
import itertools

def sitesToLinks(sites):
    links = []
    profileToOcc = {}
    linkProfiles = set()
    for g, c, p, u in sites:
        if u not in profileToOcc:
            profileToOcc[u] = {}
            
        if g not in profileToOcc[u]:
            profileToOcc[u][g] = []
            
        profileToOcc[u][g].append([g,c,p])
        
    for p in profileToOcc:
        if (len(profileToOcc[p].keys()) == 1) or (0 not in profileToOcc[p]):
            continue
            
        occs = []
        for g in profileToOcc[p]:
            occs.append(profileToOcc[p][g])
            
        l = itertools.product(*occs)
        links.extend(l)
        linkProfiles.add(p)
    
    return links, linkProfiles

links, linkProfiles = sitesToLinks(sites.numpy())
print(links[:5])
print(linkProfiles)

In [ ]:
len(links)

In [ ]:
print(insertTracking)

#### Visualize Profile Matches

In [ ]:
maxplot = 4
nplot = 0
P = specProModel.getP()
for g in list(sites.keys()):
    for c in list(sites[g].keys()):
        gseq = genomes[g][c]
        for i in range(len(sites[g][c]['profile'])):
            p = sites[g][c]['profile'][i]
            pos = sites[g][c]['pos'][i]
            score = sites[g][c]['score'][i]
            f = sites[g][c]['frame'][i]
            
            match = gseq[pos:(pos+specProModel.k*3)+2]
            match_aa = su.six_frame_translation(match)
            profile_df = su.makeDFs(P[:,:,p:(p+1)].numpy())[0]
            logo = logomaker.Logo(profile_df, vpad=.1, width=1)
            logo.style_xticks(anchor=0, spacing=1, rotation=45)
            logo.ax.set_ylabel('information (bits)')
            logo.ax.set_title(f"Profile {p}" + (f" score={score:.1f}") + " | sequence match (frame "+str(f)+") "+', '.join(match_aa))
            
            nplot += 1
            if nplot >= maxplot:
                break
                
        if nplot >= maxplot:
            break
            
    if nplot >= maxplot:
        break

In [ ]:
_, _ = printExpectedPatterns()

#### "Sensitivity" of Profile Matches

In [ ]:
def parseSequenceHeader(header):
    fields = header.split("|")
    headDict = {}
    for field in fields:
        key, value = field.split(":")
        assert len(field.split(":")) == 2, header
        assert key not in headDict, header
        headDict[key] = value

    return headDict

In [ ]:
with open(os.path.join(datapath, "../hg38.GTF.json"), 'rt') as fh:
    gtf = json.load(fh)
    
gtf['ENST00000219797'][:2]

In [ ]:
with open(os.path.join(datapath, "../orthologs.json"), 'rt') as fh:
    orthology = json.load(fh)
    
orthology[list(orthology.keys())[0]]

In [ ]:
seqnames[0][0:2]

In [ ]:
nexons = 0
northo = 0
for seq in seqnames[0]:
    seqDict = parseSequenceHeader(seq)
    if seqDict['tid'] != 'artificial':
        northo += 1
    if seqDict['tid'] in gtf:
        nexons += len([1 for annot in gtf[seqDict['tid']] if annot['feature'] == 'CDS'])
        
print(nexons)
print(northo)

In [ ]:
tolerance = 0#3*k + 3 # allow certain position mismatch in human exons
tp = [] # accumulate scores
tp_links = [] # collect links for further investigation
fp = []
noise = []
geneNoise = []
exonsFound = set()
featuresFound = set()
orthoFound = set()
for link in links:
    allArt = True
    allGen = True
    for occ in link:
        #print("DEBUG", occ)
        seqname = seqnames[occ[0]][occ[1]]
        seqDict = parseSequenceHeader(seqname)
        if seqDict['gid'] == 'artificial':
            allGen = False
        else:
            allArt = False
            
    mscore = 0#np.mean([o[3] for o in link])
    if allArt:
        fp.append(mscore)
    elif not (allArt or allGen):
        noise.append(mscore)
    else:
        assert allGen
        rSeq = seqnames[link[0][0]][link[0][1]]
        assert rSeq in orthology, rSeq
        if all([seqnames[o[0]][o[1]] in orthology[rSeq] for o in link[1:]]):
            tp.append(mscore)
            tp_links.append(link)
            orthoFound.add(rSeq)
            seqDict = parseSequenceHeader(rSeq)
            tid = seqDict['tid']
            pos = link[0][2]
            if tid in gtf:
                for feature in gtf[tid]:
                    if feature['rstart']-tolerance <= pos and pos <= feature['rend']+tolerance:
                        featuresFound.add(str(feature))
                        if feature['feature'] == 'CDS':
                            exonsFound.add(feature['tag'])
                    else:
                        print("[DEBUG]", pos, "not in", feature['rstart'], feature['rend'], "(", feature['feature'],")")
        else:
            geneNoise.append(mscore)

In [ ]:
print("TP:", len(tp), "| mean score:", sum(tp)/len(tp) if len(tp) > 0 else 0)
print("FP:", len(fp), "| mean score:", sum(fp)/len(fp) if len(fp) > 0 else 0)
print("noise:", len(noise), "| mean score:", sum(noise)/len(noise) if len(noise) > 0 else 0)
print("geneNoise:", len(geneNoise), "| mean score:", sum(geneNoise)/len(geneNoise) if len(geneNoise) > 0 else 0)

print("Sn:", len(exonsFound), "/", nexons, "=", len(exonsFound)/nexons)
print("hits:", len(orthoFound), "/", northo, "=", len(orthoFound)/northo)

In [ ]:
tp_links

In [ ]:
seqnames[0][2]

In [ ]:
for f in featuresFound:
    print(f)

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=tp, name="TP", histnorm='probability'))
fig.add_trace(go.Histogram(x=fp, name="FP", histnorm='probability'))
fig.add_trace(go.Histogram(x=noise, name="noise", histnorm='probability'))
fig.add_trace(go.Histogram(x=geneNoise, name="geneNoise", histnorm='probability'))

# Overlay both histograms and reduce opacity to see both histograms
#fig.update_layout(barmode='overlay')
#fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
assert False # stop here

In [ ]:
Q

In [ ]:
str(P[:,:,0][0].numpy())

In [ ]:
def profileToMEME(p, Q, i):
    assert p.shape[1] == len(Q), str(p.shape)+", "+str(len(Q))
    assert p.shape[1] == len(su.aa_alphabet[1:]), str(p.shape)+", "+str(len(su.aa_alphabet[1:]))
    
    text = 'MEME version 4\n'
    text += 'ALPHABET "extProtein" PROTEIN-LIKE\n'
    for aa in su.aa_alphabet[1:]:
        text += aa+"\n"
        
    text += 'END ALPHABET\n'
    text += "Background letter frequencies\n"
    freq = " ".join([su.aa_alphabet[i+1]+" "+str(Q[i]) for i in range(len(Q))])
    text += freq+"\n"
    text += "MOTIF "+str(i)+" "+str(i)+"\n"
    text += "letter-probability matrix: alength= "+str(p.shape[1])+" w= "+str(p.shape[0])+"\n"
    


    AC  MA0001.1
    XX
    ID  AGL3
    XX
    DE  MA0001.1 AGL3; from JASPAR
    PO       A     C     G     T
    1        0    94     1     2
    2        3    75     0    19
    3       79     4     3    11
    4       40     3     4    50
    5       66     1     1    29
    6       48     2     0    47
    7       65     5     5    22
    8       11     2     3    81
    9       65     3    28     1
    10       0     3    88     6
    XX
    CC  program: jaspar
    XX
    //

In [ ]:
def profileToTransfac(p, i):
    assert p.shape[1] == len(su.aa_alphabet[1:]), str(p.shape)+", "+str(len(su.aa_alphabet[1:]))
    text = "AC  "+str(i)+"\n"
    text += "XX\n"
    text += "ID  "+str(i)+"\n"
    text += "XX\n"
    text += "DE  "+str(i)+"\n"
    text += "PO  "+(" ".join([su.aa_alphabet[i+1] for i in range(len(Q))]))+"\n"
    for i in range(p.shape[0]):
        text += str(i+1)+"  "+(" ".join(str(v) for v in p.numpy()[i]))+"\n"
        
    text += "XX\n"
    text += "CC  program: own\n"
    text += "//"
    return text

In [ ]:
print(profileToTransfac(P[:,:,0], 0))

In [ ]:
with open("/nas-hs/users/matthis/genomegraph/testProfileTransfacOutput.txt", 'wt') as fh:
    for i in range(P.shape[2]):
        fh.write(profileToTransfac(P[:,:,i], i))

In [ ]:
def cosineSim(a: np.ndarray, b: np.ndarray):
    assert len(a) == len(b)
    return np.sum(a*b) / (np.sqrt(np.sum(np.square(a))) * np.sqrt(np.sum(np.square(b))))

In [ ]:
cosineSim(np.array([0,1,0,0]), np.array([1,0,0,0]))

In [ ]:
# modified version of https://gist.github.com/nornagon/6326a643fc30339ece3021013ed9b48c
# enforce gapless alignment
def smith_waterman(a, b) -> float:
    assert a.shape[1] == b.shape[1], str(a.shape)+", "+str(b.shape)
    # H holds the alignment score at each point, computed incrementally
    H = np.zeros((a.shape[0] + 1, b.shape[0] + 1))
    for i in range(1, a.shape[0] + 1):
        for j in range(1, b.shape[0] + 1):
            # The score for substituting the letter a[i-1] for b[j-1]. Generally low
            # for mismatch, high for match.
            H[i,j] = H[i-1,j-1] + cosineSim(a[i-1], b[j-1])
    # The highest score is the best local alignment.
    # For our purposes, we don't actually care _what_ the alignment was, just how
    # aligned the two strings were.
    return H.max()

In [ ]:
def profileLogo(i, title="", P=P):
    profile_df = su.makeDFs(P[:,:,i:(i+1)].numpy())[0]
    logo = logomaker.Logo(profile_df, vpad=.1, width=1)
    logo.style_xticks(anchor=0, spacing=1, rotation=45)
    logo.ax.set_ylabel('information (bits)')
    logo.ax.set_title(title)

In [ ]:
a = P[:,:,0]
profileLogo(0, "Reference profile")

for i in range(5653,5658):
    b = P[:,:,i]
    s = smith_waterman(a,b)
    profileLogo(i, "Aligning with score "+str(s))

In [ ]:
for i in range(P.shape[2]):
    for j in range(i+1, P.shape[2]):
        if smith_waterman(P[:,:,i], P[:,:,j]) >= P.shape[0]/2:
            print("Similar:",i,"and",j)
            profileLogo(i,str(i))
            profileLogo(j,str(j))

---

## Find Best Profiles

Idea: 
* convert profiles into aa-sequences (translate max. values into aa at each position)
* align (or perfect match?) sequences against the 6 possible pattern translations and the insert translations
* alignment scores give an idea of how good the profiles are -> allows multiple training runs to account for randomness, evaluate how good the profiles usually are (+ variance)

In [ ]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [ ]:
def runTraining():
    tf.keras.backend.clear_session() # avoid memory cluttering by remains of old models
    
    # build and randomly initialize profile model
    specProModel = SpecificProfile(k, su.aa_alphabet_size, U, Q)
    
    #start = time()
    specProModel.train_ds(ds, steps_per_epoch, epochs=500, verbose=False)
    #end = time()
    #print(f"time: {end-start:.2f}")
    
    #S, R = specProModel(X)
    #_, (L1, L2, L4, L7) = specProModel.loss(S)
    #pScores = tf.reduce_sum(tf.reduce_max(S,axis=0), axis=0).numpy()
    #print("pScores", pScores)
    #np.argsort(pScores)
    #print ("loss=", L1.numpy(), "\tL2=", L2.numpy(), 
    #       "\tL4=", L4.numpy(), "\nS=", S.numpy())
    
    P = specProModel.getP()
    
    return P

In [ ]:
def aaFromProfiles(P):
    profiles_aa = []
    for pidx in range(P.shape[2]):
        pmat = P[:,:,pidx]
        profiles_aa.append(su.to_aa_seq(pmat))

    return profiles_aa

In [ ]:
def getBestAlignments(profiles, patterns, scoreThreshold = int(k/2)):
    scores = []
    alignments = []
    best_profiles = []
    for profile in profiles:
        pscores = []
        palignments = []
        for pattern in patterns:
            als = pairwise2.align.localxd(profile, pattern, openA=-1000, extendA=-1000, openB=-1000, extendB=-1000)
            s = [al.score for al in als]
            if len(s) > 0:
                pscores.append(np.argmax(s))
                palignments.append(als[np.argmax(s)])

        best_al = palignments[np.argmax(pscores)]
        if best_al.score >= scoreThreshold:
            scores.append(best_al.score)
            alignments.append(best_al)
            best_profiles.append(profile)

    if len(scores) > 0:
        sortedIdx = np.argsort([-s for s in scores])
        scores = [scores[i] for i in sortedIdx]
        alignments = [alignments[i] for i in sortedIdx]
        best_profiles = [best_profiles[i] for i in sortedIdx]
    
    return scores, alignments, best_profiles

In [ ]:
if False:
    for i in range(20):
        P = runTraining()
        profiles = aaFromProfiles(P)
        print("Run", i)
        if insertPatterns is not None:
            print("Desired profiles:")
            patternScores, patternAlignments, patternProfiles = getBestAlignments(profiles, desiredPatternAA)
            for i in range(len(patternScores)):
                    print(patternProfiles[i])
                    print(format_alignment(*(patternAlignments[i])))

        if repeatPatterns is not None:
            print("Undesired profiles:")
            repeatScores, repeatAlignments, repeatProfiles = getBestAlignments(profiles, repeatPatternAA)
            for i in range(len(repeatScores)):
                    print(repeatProfiles[i])
                    print(format_alignment(*(repeatAlignments[i])))

        print("")
        print("-----")
        print("")
        
else:
    P = specProModel.getP()
    profiles = aaFromProfiles(P)

    if insertPatterns is not None:
        print("Desired profiles:")
        patternScores, patternAlignments, patternProfiles = getBestAlignments(profiles, desiredPatternAA, scoreThreshold=4)
        for i in range(len(patternScores)):
                print(patternProfiles[i])
                print(format_alignment(*(patternAlignments[i])))

    if repeatPatterns is not None:
        print("Undesired profiles:")
        repeatScores, repeatAlignments, repeatProfiles = getBestAlignments(profiles, repeatPatternAA, scoreThreshold=4)
        for i in range(len(repeatScores)):
            print(repeatProfiles[i])
            print(format_alignment(*(repeatAlignments[i])))

In [ ]:
R = specProModel.getR()
profiles = aaFromProfiles(R)
if insertPatterns is not None:
    print("Desired profiles:")
    patternScores, patternAlignments, patternProfiles = getBestAlignments(profiles, desiredPatternAA, scoreThreshold=4)
    for i in range(len(patternScores)):
            print(patternProfiles[i])
            print(format_alignment(*(patternAlignments[i])))

if repeatPatterns is not None:
    print("Undesired profiles:")
    repeatScores, repeatAlignments, repeatProfiles = getBestAlignments(profiles, repeatPatternAA, scoreThreshold=4)
    for i in range(len(repeatScores)):
        print(repeatProfiles[i])
        print(format_alignment(*(repeatAlignments[i])))

---

### Evaluate Parameter Grid

In [ ]:
from datetime import time, datetime
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
dfN = pd.read_csv("20211111_evaluate_gensize_N.csv")
dfGS = pd.read_csv("20211111_evaluate_gensize_GS.csv")
dfNS = pd.read_csv("20211111_evaluate_gensize_NS.csv")
print(dfN)
print(dfGS)
print(dfNS)

In [ ]:
def timeFromStr(tstr: str):
    h,m,s = tstr.split(":")
    s, us = s.split(".")
    us = float("0."+us) * 1000000
    #print(h,m,s,us)
    return time(int(h),int(m),int(s), int(us))

def secondsFromStr(tstr: str):
    h,m,s = tstr.split(":")
    seconds = (int(h)*60*60) + (int(m)*60) + float(s)
    return seconds
    
runtime = [secondsFromStr(s) for s in df['runtime']]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['U'], y=runtime, name="runtime"))
fig.update_xaxes(title_text='U')
fig.update_yaxes(title_text='runtime [s]')#, type='log')
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['U'], y=df['memory'], name="memory [MB]"))
fig.update_xaxes(title_text='U')
fig.update_yaxes(title_text='Memory [MB]')
fig.show()

In [ ]:
gsN = []
for _, row in dfN.iterrows():
    gsN.append(int(row['N']) * int(row['genome-sizes']))
    
gsGS = []
for _, row in dfGS.iterrows():
    gsGS.append(int(row['N']) * int(row['genome-sizes']))
    
gsNS = []
for _, row in dfNS.iterrows():
    gs = [int(s) for s in row['genome-sizes'].split()]
    gsNS.append(int(row['N']) * sum(gs))

fig = go.Figure()
fig.add_trace(go.Scatter(x=gsN, y=[secondsFromStr(s) for s in dfN['runtime']], name="runtime N"))
fig.add_trace(go.Scatter(x=gsGS, y=[secondsFromStr(s) for s in dfGS['runtime']], name="runtime GS"))
fig.add_trace(go.Scatter(x=gsNS, y=[secondsFromStr(s) for s in dfNS['runtime']], name="runtime NS"))
fig.update_xaxes(title_text='genome size')
fig.update_yaxes(title_text='runtime [s]')
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=gsN, y=dfN['memory'], name="memory [MB] N"))
fig.add_trace(go.Scatter(x=gsGS, y=dfGS['memory'], name="memory [MB] GS"))
fig.add_trace(go.Scatter(x=gsNS, y=dfNS['memory'], name="memory [MB] NS"))
fig.update_xaxes(title_text='genome size')
fig.update_yaxes(title_text='Memory [MB]')
fig.show()

In [ ]:
x = []
yr = []
ym = []
for _, row in dfN.iterrows():
    x.append(int(row['N']))
    yr.append(secondsFromStr(row['runtime']))
    ym.append(float(row['memory']))
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=yr, name="runtime"))
fig.update_xaxes(title_text='genome size')
fig.update_yaxes(title_text='runtime [s]')
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=ym, name="memory [MB] N"))
fig.update_xaxes(title_text='genome size')
fig.update_yaxes(title_text='Memory [MB]')
fig.show()

In [ ]:
x = []
yr = []
ym = []
for _, row in dfGS.iterrows():
    x.append(int(row['genome-sizes']))
    yr.append(secondsFromStr(row['runtime']))
    ym.append(float(row['memory']))
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=yr, name="runtime"))
fig.update_xaxes(title_text='genome size')
fig.update_yaxes(title_text='runtime [s]')
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=ym, name="memory [MB] N"))
fig.update_xaxes(title_text='genome size')
fig.update_yaxes(title_text='Memory [MB]')
fig.show()

In [ ]:
x = []
yr = []
ym = []
for _, row in dfNS.iterrows():
    x.append(sum([int(s) for s in row['genome-sizes'].split()]))
    yr.append(secondsFromStr(row['runtime']))
    ym.append(float(row['memory']))
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=yr, name="runtime"))
fig.update_xaxes(title_text='genome size')
fig.update_yaxes(title_text='runtime [s]')
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=ym, name="memory [MB] N"))
fig.update_xaxes(title_text='genome size')
fig.update_yaxes(title_text='Memory [MB]')
fig.show()